In [ ]:
import pandas as pd
import numpy as np

# ====================== CONFIGURABLE PARAMETERS ======================
# Paths to model prediction files
file_paths = [
    "your_result_path1/submission.csv",
    "your_result_path2/submission.csv",
    "your_result_path3/submission.csv"
]

# Voting threshold (number of models that must predict 1 for the final result to be 1)
VOTE_THRESHOLD = 2

# Path to save the final submission file
OUTPUT_PATH = "your_ensemble_result_path/submission.csv"
# ========================================================

# Read prediction files and fill missing values with an empty string
dfs = [pd.read_csv(fp).fillna('') for fp in file_paths]

# Function to encode a binary mask into RLE (Run-Length Encoding)
def mask2rle(img):
    pixels = img.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

# Function to decode RLE into a binary mask
def rle2mask(mask_rle, shape=(256, 256)):
    if not mask_rle:
        return np.zeros(shape, dtype=np.uint8)
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0] * shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape)

# Decode RLE-encoded predictions into binary masks
masks = []
for df in dfs:
    masks.append({row['ImageId']: rle2mask(row['EncodedPixels']) for _, row in df.iterrows()})

# Perform majority voting to generate the final mask
final_masks = {}
for image_id in masks[0]:
    # Sum predictions from all models and apply the voting threshold
    combined = sum(mask[image_id] for mask in masks)
    final_masks[image_id] = (combined >= VOTE_THRESHOLD).astype(np.uint8)

# Encode final masks using RLE format
submission = pd.DataFrame({
    'ImageId': list(final_masks.keys()),
    'EncodedPixels': [mask2rle(final_masks[image_id]) for image_id in final_masks]
})

# Save the final submission file
submission.to_csv(OUTPUT_PATH, index=False)

print(f"Submission saved to {OUTPUT_PATH}")
